In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
from pathlib import Path
from IPython.display import display

import torch
import torch_geometric.data as tgd
import rpad.partnet_mobility_utils.dataset as rpd

import flowbot3d.tg_dataset as ftd
import flowbot3d.models.flowbot3d as fmf

# Make a prediction, and visualize the results.


In [ ]:
# Create the dataset.
dataset = ftd.Flowbot3DTGDataset(
    root=os.path.expanduser("~/datasets/partnet-mobility/raw/"),
    split="umpnet-train-test",
    randomize_camera=False,
)


# Load the model.
ckpt_path = "../checkpoints/wandb/vermilion-lantern-25/epoch=99-step=78600.ckpt"
model = fmf.ArtFlowNet.load_from_checkpoint(ckpt_path).cuda()
model.eval()

# Run inference on a single example.
data = dataset.get_data("100561")
batch = tgd.Batch.from_data_list([data])
with torch.no_grad():
    pred_flow = model(batch.cuda()).cpu()

# Display the figure.
fig = fmf.ArtFlowNet.make_plots(pred_flow, batch.cpu())["artflownet_plot"]
fig.update_layout(autosize=False, height=800, width=1200)
fig

# Metrics for various ablations.

In [ ]:
def make_tables(results_dir, method_names, metric, method_display_names=None, higherbetter=True):
    dfs = []
    if method_display_names is None:
        method_display_names = method_names
    for split in ['umpnet-train-train', 'umpnet-train-test', 'umpnet-test']:
        rows = []
        for method_name in method_names:
            csv = Path(results_dir) / method_name / f"{split}.csv"
            df = pd.read_csv(csv, index_col=0)
            rows.append(df.loc[metric])
        df = pd.DataFrame(rows, method_display_names)
        df = df.style.set_caption(split)
        if higherbetter:
            df = df.highlight_max(color="lightblue")
        else:
            df = df.highlight_min(color="lightblue")
        dfs.append(df)
    return dfs
        

## RMSE

In [ ]:
results_dir = "../results"
method_names = ["f3d_mask_novpa", "f3d_nomask_vpa", "f3d_nomask_novpa"]
method_display_names = ["FlowBot3D", "FlowBot3D w/o Mask", "FlowBot3D w/o Mask (+VPA)"]
metric = "rmse"
dfs = make_tables("../results", method_names, metric, method_display_names, higherbetter=False)
for df in dfs:
    display(df)

## Cosine Distance

In [ ]:
results_dir = "../results"
method_names = ["f3d_mask_novpa", "f3d_nomask_vpa", "f3d_nomask_novpa"]
method_display_names = ["FlowBot3D", "FlowBot3D w/o Mask", "FlowBot3D w/o Mask (+VPA)"]
metric = "cos_dist"
dfs = make_tables("../results", method_names, metric, method_display_names, higherbetter=True)
for df in dfs:
    display(df)

## Flow Magnitude Error

In [ ]:
results_dir = "../results"
method_names = ["f3d_mask_novpa", "f3d_nomask_vpa", "f3d_nomask_novpa"]
method_display_names = ["FlowBot3D", "FlowBot3D w/o Mask", "FlowBot3D w/o Mask (+VPA)"]
metric = "mag_error"
dfs = make_tables("../results", method_names, metric, method_display_names, higherbetter=False)
for df in dfs:
    display(df)